In [179]:
%load_ext autoreload

import graphistry
import neo4j
import json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [180]:
def get_graph(a, b = {}):
    driver = neo4j.GraphDatabase.driver("bolt://neo4j:7687")
    # MATCH (a)-[r:TRANSACTION]->(b) WHERE r.USD > 7000 AND r.USD < 10000  RETURN a, r, b ORDER BY r.USD DESC LIMIT 5
    with driver.session() as session:
        return session.run(a, b).graph()

In [181]:
# graph = get_graph("MATCH (a)-[r:PAYMENT]->(b) WHERE r.USD > 7000 AND r.USD < 10000 RETURN * LIMIT 5000")
graph = get_graph("""
    match (a)-[r:PAYMENT*1..20]->(b) 
    where a.Account = $root and ALL(transfer IN r WHERE transfer.USD > $min_amount and transfer.USD < $max_amount )
    return a, r, b
  """, 
  {'root': "Carl Force (DEA)", 
   'min_amount': 999, 
   'max_amount': 99999})

In [182]:
%autoreload
plotter = graphistry \
    .data(graph=graph) \
    .bind(
        node_id='__node_id__',
        edge_id='__edge_id__',
        edge_src='__edge_src__',
        edge_dst='__edge_dst__'
    )

In [183]:
plotter.plot()

Exception: column types mismatch (Account/__edge_src__). expected(string) actual(int64)

In [173]:
graph = get_graph("""
    match (a)-[r:PAYMENT*1..20]->(b) 
    where a.Account = $root and ALL(transfer IN r WHERE transfer.USD > $min_amount and transfer.USD < $max_amount )
    return a, r, b
  """, 
  {'root': "Carl Force (DEA)", 
   'min_amount': 999, 
   'max_amount': 99999})